# Chapter 13: Texturing

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/computer-vision/blob/main/chapter_13_texturing.ipynb)

**Texturing** adds surface detail to 3D models by mapping 2D images onto 3D geometry. This chapter covers UV mapping, texture filtering, mipmapping, and various texture types used in modern rendering.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from typing import Tuple, List, Optional
from PIL import Image

In [ ]:
class Vec2:
    """2D Vector for texture coordinates"""
    def __init__(self, x: float = 0.0, y: float = 0.0):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        return Vec2(self.x + other.x, self.y + other.y)
    
    def __mul__(self, scalar: float):
        return Vec2(self.x * scalar, self.y * scalar)

class Vec3:
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x, self.y, self.z = x, y, z
    
    def __add__(self, other):
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    def __sub__(self, other):
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __mul__(self, scalar: float):
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def dot(self, other) -> float:
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    def normalize(self):
        length = math.sqrt(self.dot(self))
        if length > 0:
            return Vec3(self.x / length, self.y / length, self.z / length)
        return Vec3(0, 0, 0)

print("✓ Base classes loaded")

## 1. UV Mapping

**UV mapping** assigns 2D texture coordinates $(u, v)$ to 3D vertices.

### Coordinate Space

- $(u, v) \in [0, 1] \times [0, 1]$
- $(0, 0)$ = bottom-left of texture
- $(1, 1)$ = top-right of texture

### Parametric Surfaces

**Sphere UV mapping**:
$$
u = 0.5 + \frac{\arctan2(z, x)}{2\pi}
$$
$$
v = 0.5 - \frac{\arcsin(y)}{\pi}
$$

**Cylinder UV mapping**:
$$
u = \frac{\arctan2(z, x)}{2\pi}
$$
$$
v = y
$$

**Plane UV mapping**:
$$
u = x, \quad v = z
$$

In [ ]:
def sphere_uv(point: Vec3) -> Vec2:
    """Calculate UV coordinates for sphere"""
    # Normalize to unit sphere
    p = point.normalize()
    
    # Spherical coordinates
    phi = math.atan2(p.z, p.x)
    theta = math.asin(p.y)
    
    u = 0.5 + phi / (2 * math.pi)
    v = 0.5 - theta / math.pi
    
    return Vec2(u, v)

def cylinder_uv(point: Vec3, height: float) -> Vec2:
    """Calculate UV coordinates for cylinder"""
    phi = math.atan2(point.z, point.x)
    u = phi / (2 * math.pi)
    v = point.y / height
    return Vec2(u, v)

def plane_uv(point: Vec3, scale: float = 1.0) -> Vec2:
    """Calculate UV coordinates for plane"""
    u = point.x * scale
    v = point.z * scale
    return Vec2(u, v)

print("✓ UV mapping functions loaded")

## 2. Texture Sampling

### Nearest Neighbor (Point Sampling)

$$
C(u, v) = T[\lfloor u \cdot w \rfloor, \lfloor v \cdot h \rfloor]
$$

Fast but produces blocky artifacts.

### Bilinear Interpolation

$$
C(u, v) = (1-t_x)(1-t_y) \cdot C_{00} + t_x(1-t_y) \cdot C_{10} + (1-t_x)t_y \cdot C_{01} + t_x t_y \cdot C_{11}
$$

where:
- $t_x = \text{frac}(u \cdot w)$
- $t_y = \text{frac}(v \cdot h)$
- $C_{ij}$ are the four nearest texels

### Wrapping Modes

- **Repeat**: $u' = u \mod 1$
- **Clamp**: $u' = \text{clamp}(u, 0, 1)$
- **Mirror**: $u' = 1 - |2(u \mod 2) - 1|$

In [ ]:
class Texture:
    """2D texture with sampling methods"""
    def __init__(self, width: int, height: int, data: np.ndarray = None):
        self.width = width
        self.height = height
        if data is None:
            self.data = np.zeros((height, width, 3), dtype=np.float32)
        else:
            self.data = data.astype(np.float32)
            if len(self.data.shape) == 2:
                self.data = np.stack([self.data] * 3, axis=-1)
    
    @staticmethod
    def from_image(path: str):
        """Load texture from image file"""
        img = Image.open(path).convert('RGB')
        data = np.array(img) / 255.0
        return Texture(img.width, img.height, data)
    
    @staticmethod
    def checkerboard(width: int, height: int, freq: int = 8) -> 'Texture':
        """Create checkerboard texture"""
        data = np.zeros((height, width, 3), dtype=np.float32)
        for j in range(height):
            for i in range(width):
                check = ((i // (width // freq)) + (j // (height // freq))) % 2
                data[j, i] = [1.0, 1.0, 1.0] if check else [0.2, 0.2, 0.2]
        return Texture(width, height, data)
    
    @staticmethod
    def gradient(width: int, height: int) -> 'Texture':
        """Create gradient texture"""
        data = np.zeros((height, width, 3), dtype=np.float32)
        for j in range(height):
            for i in range(width):
                u = i / width
                v = j / height
                data[j, i] = [u, v, (u + v) / 2]
        return Texture(width, height, data)
    
    def wrap_repeat(self, coord: float) -> float:
        """Repeat wrapping mode"""
        return coord - math.floor(coord)
    
    def wrap_clamp(self, coord: float) -> float:
        """Clamp wrapping mode"""
        return max(0.0, min(1.0, coord))
    
    def sample_nearest(self, u: float, v: float) -> Tuple[float, float, float]:
        """Nearest neighbor sampling"""
        u = self.wrap_repeat(u)
        v = self.wrap_repeat(v)
        
        x = int(u * self.width) % self.width
        y = int(v * self.height) % self.height
        
        return tuple(self.data[y, x])
    
    def sample_bilinear(self, u: float, v: float) -> Tuple[float, float, float]:
        """Bilinear interpolation sampling"""
        u = self.wrap_repeat(u)
        v = self.wrap_repeat(v)
        
        x_f = u * self.width
        y_f = v * self.height
        
        x0 = int(math.floor(x_f)) % self.width
        x1 = (x0 + 1) % self.width
        y0 = int(math.floor(y_f)) % self.height
        y1 = (y0 + 1) % self.height
        
        tx = x_f - math.floor(x_f)
        ty = y_f - math.floor(y_f)
        
        c00 = self.data[y0, x0]
        c10 = self.data[y0, x1]
        c01 = self.data[y1, x0]
        c11 = self.data[y1, x1]
        
        color = (1 - tx) * (1 - ty) * c00 + \
                tx * (1 - ty) * c10 + \
                (1 - tx) * ty * c01 + \
                tx * ty * c11
        
        return tuple(color)

print("✓ Texture class loaded")

## 3. Mipmapping

**Mipmaps** are pre-filtered texture pyramid to prevent aliasing:

$$
\text{Level } d: \quad \text{size} = \frac{\text{original size}}{2^d}
$$

### Level Selection

Mipmap level based on texture coordinate derivatives:

$$
d = \log_2\left(\max\left(\sqrt{\left(\frac{\partial u}{\partial x}\right)^2 + \left(\frac{\partial v}{\partial x}\right)^2}, \sqrt{\left(\frac{\partial u}{\partial y}\right)^2 + \left(\frac{\partial v}{\partial y}\right)^2}\right) \cdot \text{tex\_size}\right)
$$

### Trilinear Filtering

Interpolate between two mipmap levels:

$$
C = (1 - t_d) \cdot \text{bilinear}(d_0) + t_d \cdot \text{bilinear}(d_1)
$$

In [ ]:
class MipmappedTexture:
    """Texture with mipmap levels"""
    def __init__(self, base_texture: Texture):
        self.levels = [base_texture]
        self._generate_mipmaps()
    
    def _generate_mipmaps(self):
        """Generate mipmap pyramid"""
        current = self.levels[0]
        
        while current.width > 1 and current.height > 1:
            new_width = max(1, current.width // 2)
            new_height = max(1, current.height // 2)
            
            new_data = np.zeros((new_height, new_width, 3), dtype=np.float32)
            
            for j in range(new_height):
                for i in range(new_width):
                    # Average 2x2 block
                    y0, y1 = j * 2, min(j * 2 + 1, current.height - 1)
                    x0, x1 = i * 2, min(i * 2 + 1, current.width - 1)
                    
                    avg = (current.data[y0, x0] + current.data[y0, x1] +
                          current.data[y1, x0] + current.data[y1, x1]) / 4.0
                    
                    new_data[j, i] = avg
            
            current = Texture(new_width, new_height, new_data)
            self.levels.append(current)
    
    def sample(self, u: float, v: float, lod: float = 0.0) -> Tuple[float, float, float]:
        """Sample with level of detail"""
        lod = max(0, min(lod, len(self.levels) - 1))
        level = int(math.floor(lod))
        
        if level >= len(self.levels) - 1:
            return self.levels[-1].sample_bilinear(u, v)
        
        # Trilinear filtering
        c0 = self.levels[level].sample_bilinear(u, v)
        c1 = self.levels[level + 1].sample_bilinear(u, v)
        
        t = lod - level
        color = tuple((1 - t) * c0[i] + t * c1[i] for i in range(3))
        
        return color

print("✓ Mipmap class loaded")

## 4. Texture Types

### Diffuse/Albedo Maps
Base color of the surface.

### Normal Maps
Perturb surface normals for detail:
$$
\mathbf{n}' = TBN \cdot \mathbf{n}_{map}
$$

where $TBN$ is the tangent-bitangent-normal matrix.

### Specular Maps
Control specular reflectance.

### Roughness Maps
Control surface roughness (PBR).

### Ambient Occlusion Maps
Simulate crevice darkening.

### Displacement/Height Maps
Actually modify geometry.

## Example 1: Texture Filtering Comparison

In [ ]:
# Create checkerboard texture
tex = Texture.checkerboard(128, 128, freq=8)

# Sample with different methods
width, height = 256, 256
nearest_img = np.zeros((height, width, 3))
bilinear_img = np.zeros((height, width, 3))

for j in range(height):
    for i in range(width):
        u = (i / width) * 4  # Magnify
        v = (j / height) * 4
        
        nearest_img[j, i] = tex.sample_nearest(u, v)
        bilinear_img[j, i] = tex.sample_bilinear(u, v)

# Display
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(tex.data)
axes[0].set_title('Original Texture')
axes[0].axis('off')

axes[1].imshow(nearest_img)
axes[1].set_title('Nearest Neighbor (Blocky)')
axes[1].axis('off')

axes[2].imshow(bilinear_img)
axes[2].set_title('Bilinear (Smooth)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Notice: Bilinear filtering produces smoother magnification.")

## Example 2: Mipmap Visualization

In [ ]:
# Create gradient texture
base = Texture.gradient(256, 256)
mipmap = MipmappedTexture(base)

# Visualize mipmap levels
num_levels = min(5, len(mipmap.levels))
fig, axes = plt.subplots(1, num_levels, figsize=(15, 3))

for i in range(num_levels):
    axes[i].imshow(mipmap.levels[i].data)
    axes[i].set_title(f'Level {i} ({mipmap.levels[i].width}x{mipmap.levels[i].height})')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

print(f"Generated {len(mipmap.levels)} mipmap levels.")

## Example 3: Textured Sphere

In [ ]:
# Create procedural texture
tex_sphere = Texture.checkerboard(256, 128, freq=16)

# Render textured sphere
width, height = 400, 400
image = np.zeros((height, width, 3))

sphere_center = Vec3(0, 0, 0)
sphere_radius = 1.5
light_dir = Vec3(1, 1, 1).normalize()

for j in range(height):
    for i in range(width):
        # NDC coordinates
        x = (2.0 * i / width - 1.0) * 2
        y = (1.0 - 2.0 * j / height) * 2
        
        # Ray-sphere intersection
        ray_origin = Vec3(x, y, 5)
        ray_dir = Vec3(0, 0, -1)
        
        oc = ray_origin - sphere_center
        a = ray_dir.dot(ray_dir)
        b = 2.0 * oc.dot(ray_dir)
        c = oc.dot(oc) - sphere_radius * sphere_radius
        
        discriminant = b * b - 4 * a * c
        
        if discriminant >= 0:
            t = (-b - math.sqrt(discriminant)) / (2 * a)
            if t > 0:
                hit_point = ray_origin + ray_dir * t
                normal = (hit_point - sphere_center).normalize()
                
                # Get UV coordinates
                uv = sphere_uv(hit_point - sphere_center)
                
                # Sample texture
                tex_color = tex_sphere.sample_bilinear(uv.x, uv.y)
                
                # Simple shading
                diffuse = max(0, normal.dot(light_dir))
                shaded = tuple(tex_color[k] * (0.3 + 0.7 * diffuse) for k in range(3))
                
                image[j, i] = shaded
            else:
                image[j, i] = (0.1, 0.1, 0.15)
        else:
            image[j, i] = (0.1, 0.1, 0.15)

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title('Textured Sphere with UV Mapping')
plt.axis('off')
plt.tight_layout()
plt.show()

print("Sphere with checkerboard texture using spherical UV mapping.")

## Summary

**Texturing** adds rich surface detail to 3D models:

### Key Concepts

1. **UV Mapping**: 2D parameterization of 3D surfaces
2. **Texture Sampling**: Point, bilinear, trilinear filtering
3. **Mipmapping**: Pre-filtered texture pyramid for anti-aliasing
4. **Texture Types**: Diffuse, normal, specular, roughness, AO, displacement

### Filtering Methods

- **Nearest Neighbor**: Fast, blocky
- **Bilinear**: Smooth, some blurring
- **Trilinear**: Smooth across mipmap levels
- **Anisotropic**: Best quality, handles oblique angles

### Applications

- Surface appearance (color, roughness)
- Fine geometric detail (normal/bump mapping)
- Environmental effects (dirt, scratches)
- Performance optimization (LOD)

Texturing is essential for realistic and efficient 3D rendering in games, films, and visualization.